<a href="https://colab.research.google.com/github/Steven-Edward-Otoko/Astronomical-seeing/blob/main/Reading_the_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wrf-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.8/935.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 14.2 MB/s eta 0:00:00
  Created wheel for wrf-python: filename=wrf_python-1.3.4.1-cp310-cp310-linux_x86_64.whl size=630218 sha256=1de1b1f843921833e52cd694a34b06f179ca3b06f0f74ec43737f3f5a2296402
  Stored in directory: /root/.cache/pip/wheels/67/e1/53/9fa18acf00e88c15b3038e736ce7cd23cfb37361a2564d4db5
Successfully built wrf-python


In [ ]:

pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.3 MB/s eta 0:00:00


In [ ]:
import glob
from tqdm import tqdm
import pandas as pd
from datetime import datetime, timezone
from netCDF4 import Dataset
from wrf import getvar
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#how we iterate the data in the WRF files
"""
i=55
j=i
all_files=glob.glob("./wrf1/wrf/*/*")
dfs=[]
with tqdm(total=len(all_files)) as pbar:
    for file in all_files:
        _df=pd.DataFrame()
        date=datetime.strptime(file.split("\\")[-1],"wrfout_d03_%Y-%m-%d_%H-%M-%S")
        date.replace(tzinfo=timezone.utc)
        try:
            ncfile=Dataset(file)
        except:
            print("Error reading the file ",file)
            continue
        for var in ["z","p","avo","eth","dbz","pvo","rh","temp","tv","wspd","wdir"]:
            f=getvar(ncfile,var)
            _df[var]=f[:,i,j]
        _df["date"]=date
        dfs.append(_df)
        pbar.update(1)
df=pd.concat(dfs)
del dfs
df.to_csv("ao_as.csv",index=False)
"""


In [ ]:
#reading the data
file_path = '/content/drive/My Drive/Workshop/ao_as.csv'
df=pd.read_csv(file_path)
df.describe()
df["date"]=pd.to_datetime(df["date"])
df.tail()

,z,p,avo,eth,dbz,pvo,rh,temp,tv,wspd,wdir,date
245999,16638.848,10090.8620,5.013207,322.41864,-30.0,5.331585,100.000000,167.42111,167.42140,8.527791,246.086170,2023-08-29 06:00:00
246000,17551.371,8638.6530,4.882956,748.17065,-30.0,5.321727,0.000000,371.62976,371.63030,3.703524,246.075200,2023-08-29 06:00:00
246001,18473.023,7395.6290,6.095748,469.13425,-30.0,12.850584,0.444598,222.91090,222.91122,2.839859,11.563299,2023-08-29 06:00:00
246002,19406.100,6331.4634,5.400429,944.83075,-30.0,16.327408,0.000000,429.44788,429.44846,5.650360,63.441433,2023-08-29 06:00:00
246003,20352.440,5420.3320,7.515312,688.35840,-30.0,20.861956,0.000360,299.29156,299.29190,8.748821,78.518036,2023-08-29 06:00:00


In [ ]:
#data set construction
# Number of sections
d=3
df['Subgroup'] = df.groupby('date').transform(lambda x: pd.qcut(range(len(x)), q=d, labels=False, duplicates='drop')).values[:,0]
DF= df.groupby(['date', 'Subgroup'])
# columns to aggregate
columns_to_aggregate = ['z', 'p', 'avo', 'eth', 'pvo', 'rh', 'temp', 'tv', 'wspd','wdir']
#operation to be calculated
op_c=['mean','std']
# Create a dictionary for aggregation functions
aggregation_dict = {col: op_c for col in columns_to_aggregate}
# Calculate mean and std for each group
DF = DF.agg(aggregation_dict)
DF.reset_index( inplace=True)
DF.columns = ['date',"Subgroup"]+['{}_{}'.format(col[0], col[1]) for col in DF.columns[2::]]
DF.tail()

,date,Subgroup,z_mean,z_std,p_mean,p_std,avo_mean,avo_std,eth_mean,eth_std,...,rh_mean,rh_std,temp_mean,temp_std,tv_mean,tv_std,wspd_mean,wspd_std,wdir_mean,wdir_std
14869,2023-08-29 05:00:00,1,5301.707543,1199.713334,54575.302929,7947.204739,8.592784,8.835831,307.389962,81.425142,...,38.268726,47.890512,255.728861,66.666733,255.829015,66.696012,3.381774,1.142189,203.146026,145.949702
14870,2023-08-29 05:00:00,2,13913.134633,4038.844032,18576.257007,10908.590657,2.777916,5.205585,513.205877,185.729017,...,20.130141,41.337399,292.920362,74.672385,292.930263,74.670951,11.150936,5.227830,218.628958,86.689063
14871,2023-08-29 06:00:00,0,2832.516127,410.841836,73175.140533,3539.420624,48.154130,57.579984,286.168096,62.770458,...,49.461102,45.608864,256.829211,56.139105,257.106165,56.205239,1.972888,1.746951,128.049971,144.026433
14872,2023-08-29 06:00:00,1,5300.120521,1199.420214,54598.456643,7957.953174,5.897816,11.805415,307.070897,81.395340,...,38.441436,47.774888,255.409423,66.625071,255.514182,66.655267,3.644822,1.188793,186.822031,146.934283
14873,2023-08-29 06:00:00,2,13912.242133,4041.497835,18576.287027,10909.504277,2.875510,3.055812,513.940860,186.905875,...,20.134605,41.335173,293.241813,74.979200,293.252080,74.977735,11.038071,5.408561,217.288379,87.576355


In [ ]:
# Define a function to map month to season
def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter'


In [ ]:
DFF = pd.pivot_table(DF, values=DF.columns[2::], index=['date'], columns=['Subgroup'])
DFF.reset_index( inplace=True)
DFF.columns = ['date']+['{}_{}'.format(col[0], col[1]) for col in DFF.columns[1::]]
DFF['Season'] = DFF['date'].dt.month.map(get_season)
DFF.head()

,date,avo_mean_0,avo_mean_1,avo_mean_2,avo_std_0,avo_std_1,avo_std_2,eth_mean_0,eth_mean_1,eth_mean_2,...,wspd_std_0,wspd_std_1,wspd_std_2,z_mean_0,z_mean_1,z_mean_2,z_std_0,z_std_1,z_std_2,Season
0,2022-06-19 21:00:00,7.918248,5.371775,7.025254,1.832495,2.657607,3.353703,294.841451,307.208499,514.499028,...,1.409158,0.792420,8.216499,2833.016307,5264.897214,13814.275767,410.197661,1173.843532,4048.821424,Summer
1,2022-06-19 22:00:00,-8.392706,5.338772,6.998164,11.859050,9.417394,5.219002,294.431080,306.270296,514.480481,...,1.162542,1.050237,8.665415,2834.377760,5270.658114,13826.584167,410.757798,1177.467010,4050.547422,Summer
2,2022-06-19 23:00:00,43.513576,4.405839,11.066138,49.386370,5.253272,5.742793,293.940831,305.657711,514.031877,...,1.709527,0.918219,8.525288,2833.924247,5271.269429,13831.426040,410.666932,1179.127617,4050.289657,Summer
3,2022-06-20 00:00:00,-10.665649,5.544251,6.164017,29.245180,7.285478,4.169569,294.224729,305.470782,514.547098,...,2.021260,0.945104,8.060775,2832.865353,5264.202350,13821.226473,409.848234,1175.928818,4051.029362,Summer
4,2022-06-20 01:00:00,2.827458,5.249824,4.715234,18.593187,10.516341,4.968519,294.740599,305.084476,514.069261,...,1.991235,0.769808,8.181346,2833.147067,5262.711064,13809.820847,409.854425,1175.536993,4044.067766,Summer


In [ ]:
dummy_df = pd.get_dummies(DFF['Season'], prefix='Season').astype(int)
# Concatenate the original DataFrame with the dummy variables
df = pd.concat([DFF, dummy_df], axis=1)
df

,date,avo_mean_0,avo_mean_1,avo_mean_2,avo_std_0,avo_std_1,avo_std_2,eth_mean_0,eth_mean_1,eth_mean_2,...,z_mean_1,z_mean_2,z_std_0,z_std_1,z_std_2,Season,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,2022-06-19 21:00:00,7.918248,5.371775,7.025254,1.832495,2.657607,3.353703,294.841451,307.208499,514.499028,...,5264.897214,13814.275767,410.197661,1173.843532,4048.821424,Summer,0,0,1,0
1,2022-06-19 22:00:00,-8.392706,5.338772,6.998164,11.859050,9.417394,5.219002,294.431080,306.270296,514.480481,...,5270.658114,13826.584167,410.757798,1177.467010,4050.547422,Summer,0,0,1,0
2,2022-06-19 23:00:00,43.513576,4.405839,11.066138,49.386370,5.253272,5.742793,293.940831,305.657711,514.031877,...,5271.269429,13831.426040,410.666932,1179.127617,4050.289657,Summer,0,0,1,0
3,2022-06-20 00:00:00,-10.665649,5.544251,6.164017,29.245180,7.285478,4.169569,294.224729,305.470782,514.547098,...,5264.202350,13821.226473,409.848234,1175.928818,4051.029362,Summer,0,0,1,0
4,2022-06-20 01:00:00,2.827458,5.249824,4.715234,18.593187,10.516341,4.968519,294.740599,305.084476,514.069261,...,5262.711064,13809.820847,409.854425,1175.536993,4044.067766,Summer,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,2023-08-29 02:00:00,-0.664584,16.437879,3.790227,76.640602,7.180574,5.916893,285.762571,307.748144,512.514397,...,5303.790336,13913.560907,411.474044,1200.291546,4035.417408,Summer,0,0,1,0
4954,2023-08-29 03:00:00,3.688128,13.885189,4.440037,68.292975,7.226922,4.645146,285.723711,307.573916,513.820645,...,5302.878614,13917.291360,411.273231,1200.265606,4042.479830,Summer,0,0,1,0
4955,2023-08-29 04:00:00,-0.942544,11.025822,4.221804,51.444608,5.642906,5.061327,285.863492,307.431197,513.793929,...,5300.958871,13916.034467,411.092622,1199.504236,4044.298281,Summer,0,0,1,0
4956,2023-08-29 05:00:00,10.989430,8.592784,2.777916,43.618742,8.835831,5.205585,286.142207,307.389962,513.205877,...,5301.707543,13913.134633,411.147638,1199.713334,4038.844032,Summer,0,0,1,0


In [ ]:
# Astronomical seeing data manage
##Removing the outliers for the time

def sigma_c(data,column,sigma=3, max_iter=10):

    c_data = data[column].copy()

    for _ in range(max_iter):
        mean = np.mean(c_data)
        std = np.std(c_data)
        outliers = np.abs(c_data - mean) > sigma * std

        if np.sum(outliers) == 0:
            break
        c_data[outliers] = np.nan

    df= data[~np.isnan(c_data)].reset_index(drop=True)
    return df

In [ ]:
#sigma clipping the data
file_path = '/content/drive/My Drive/Workshop/seeing_measures.csv'
s=pd.read_csv(file_path)
s["time"]=pd.to_datetime(s["_time"])
data=sigma_c(s,"_value",1,1)

In [ ]:
#taking the mean for each hour
data["time"] = data["time"].dt.floor('H')
data=data.drop(columns=["_time"])
data = data.groupby("time").mean()
data.reset_index(inplace=True)
data.columns=["date","seeing"]
data

,date,seeing
0,2022-06-01 00:00:00,2.303333
1,2022-06-01 01:00:00,2.265000
2,2022-06-01 05:00:00,1.838500
3,2022-06-02 00:00:00,2.086750
4,2022-06-02 01:00:00,2.188621
...,...,...
4352,2023-08-28 02:00:00,1.232462
4353,2023-08-28 03:00:00,0.970000
4354,2023-08-28 04:00:00,0.929077
4355,2023-08-28 05:00:00,1.022462


In [ ]:
#merging the wrf variables and the seeing data
result_df = pd.merge(df, data, on='date', how='inner')
result_df.tail()


,date,avo_mean_0,avo_mean_1,avo_mean_2,avo_std_0,avo_std_1,avo_std_2,eth_mean_0,eth_mean_1,eth_mean_2,...,z_mean_2,z_std_0,z_std_1,z_std_2,Season,Season_Fall,Season_Spring,Season_Summer,Season_Winter,seeing
3729,2023-08-28 02:00:00,-1.045015,1.661387,4.901427,56.653403,22.129011,6.603186,291.501607,312.225089,514.775451,...,13930.258020,411.879522,1201.102623,4048.714562,Summer,0,0,1,0,1.232462
3730,2023-08-28 03:00:00,6.765478,19.576167,4.550707,51.436755,9.908706,6.897999,291.162449,312.157789,515.241073,...,13931.085060,411.958360,1201.388009,4048.894371,Summer,0,0,1,0,0.970000
3731,2023-08-28 04:00:00,4.133381,18.071688,1.511890,53.162911,9.695554,9.681375,290.759135,312.068949,514.699319,...,13930.194733,411.546443,1201.453158,4043.965731,Summer,0,0,1,0,0.929077
3732,2023-08-28 05:00:00,-6.143910,2.245686,4.216764,46.389355,19.352245,12.518995,290.583006,311.789721,514.601591,...,13927.233267,411.395447,1200.899888,4046.832717,Summer,0,0,1,0,1.022462
3733,2023-08-28 06:00:00,-7.902651,10.766264,9.415313,39.067110,10.169146,7.795189,289.910993,311.682414,514.479391,...,13929.896267,411.821826,1201.435325,4046.504934,Summer,0,0,1,0,1.102083


In [ ]:
file_path = '/content/drive/My Drive/Workshop/Final_Data.csv'
result_df.to_csv(file_path)
